In [1]:
import argparse
import os
import sys
import ast
from collections import defaultdict
import json
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from skimage import io
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

In [2]:
import neuroglancer
viewer = neuroglancer.Viewer()
print(viewer)

http://127.0.0.1:43189/v/ab152b65f015aeb1df2d65ae4d72840592a95fe1/


In [10]:
HOME = os.path.expanduser("~")
sys.path.append(os.path.join(HOME, 'programming', 'pipeline_utility/src'))
from abakit.lib.FileLocationManager import FileLocationManager
from abakit.lib.SqlController import SqlController
from abakit.lib.utilities_alignment import load_consecutive_section_transform, create_warp_transforms, \
    transform_create_alignment
from abakit.lib.utilities_contour import get_dense_coordinates, get_contours_from_annotations
from abakit.lib.utilities_process import SCALING_FACTOR

In [12]:
animal = 'MD589'
sqlController = SqlController(animal)
fileLocationManager = FileLocationManager(animal)
width = sqlController.scan_run.width
height = sqlController.scan_run.height
width = int(width * SCALING_FACTOR)
height = int(height * SCALING_FACTOR)
aligned_shape = np.array((width, height))
THUMBNAIL_PATH = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail')
THUMBNAILS = sorted(os.listdir(THUMBNAIL_PATH))
num_section = len(THUMBNAILS)
structure_dict = sqlController.get_structures_dict()


CSV_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/foundation_brain_annotations'
csvfile = os.path.join(CSV_PATH, f'{animal}_annotation.csv')
hand_annotations = pd.read_csv(csvfile)
hand_annotations['vertices'] = hand_annotations['vertices'] \
    .apply(lambda x: x.replace(' ', ','))\
    .apply(lambda x: x.replace('\n',','))\
    .apply(lambda x: x.replace(',]',']'))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ','))\
    .apply(lambda x: x.replace(',,', ',')).apply(lambda x: x.replace(',,', ','))
hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))

structures = list(hand_annotations['name'].unique())
section_structure_vertices = defaultdict(dict)
for structure in tqdm(structures):
    contour_annotations, first_sec, last_sec = get_contours_from_annotations(animal, structure, hand_annotations, densify=4)
    for section in contour_annotations:
        section_structure_vertices[section][structure] = contour_annotations[section][structure][1]


100%|██████████| 65/65 [00:05<00:00, 11.62it/s]


In [13]:
hand_annotations.head()

,Unnamed: 0,class,creator,edits,filename,id,label_position,name,orientation,parent_structure,resolution,section,side,side_manually_assigned,time_created,type,vertices
0,100157269992733,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-IHC69-2015.07.31-02.07.15_MD589_1_0205,100157269992733,[15317.43823807 12578.50401305],VCA,sagittal,[],raw,367,R,False,9152016033619,confirmed,"[[15450.64315379, 13041.15143737], [15292.3766..."
1,100472025966132,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0912201...",MD589-N51-2015.07.30-19.28.17_MD589_2_0152,100472025966132,[25707.69102116 17933.27685287],IO,sagittal,[],raw,260,R,False,9122016211223,confirmed,"[[25199.46145603, 17916.30221869], [24976.1090..."
2,10063296125941,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-N55-2015.07.30-19.49.42_MD589_3_0165,10063296125941,[28722.35048812 16712.94413081],LRt,sagittal,[],raw,286,R,False,9152016024654,confirmed,"[[29069.49187508, 17203.67921491], [28909.8937..."
3,101105413939417,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-N54-2015.07.30-19.44.32_MD589_1_0160,101105413939417,[28529.80042174 15814.38685363],LRt,sagittal,[],raw,276,R,False,9152016024654,confirmed,"[[29219.12776439, 16422.80664145], [29084.8059..."
4,101214741356495,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0531201...",MD589-N37-2015.07.30-18.32.51_MD589_1_0109,101214741356495,[22896.75853307 6674.71411486],SC,sagittal,[],raw,202,S,False,5312018021355,intersected,"[[21584.44352233, 7847.78193722], [21497.56913..."


In [16]:
##### Reproduce create_clean transform
section_offset = {}
for file_name in tqdm(THUMBNAILS):
    filepath = os.path.join(THUMBNAIL_PATH, file_name)
    img = io.imread(filepath)
    section = int(file_name.split('.')[0])
    section_offset[section] = (aligned_shape - img.shape[:2][::-1]) // 2


##### Reproduce create_alignment transform
CLEANED = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail_cleaned')

image_name_list = sorted(os.listdir(CLEANED))
anchor_idx = len(image_name_list) // 2
transformation_to_previous_sec = {}

for i in range(1, len(image_name_list)):
    fixed_fn = os.path.splitext(image_name_list[i - 1])[0]
    moving_fn = os.path.splitext(image_name_list[i])[0]
    transformation_to_previous_sec[i] = load_consecutive_section_transform(animal, moving_fn, fixed_fn)

transformation_to_anchor_sec = {}
# Converts every transformation
for moving_idx in range(len(image_name_list)):
    if moving_idx == anchor_idx:
        transformation_to_anchor_sec[image_name_list[moving_idx]] = np.eye(3)
    elif moving_idx < anchor_idx:
        T_composed = np.eye(3)
        for i in range(anchor_idx, moving_idx, -1):
            T_composed = np.dot(np.linalg.inv(transformation_to_previous_sec[i]), T_composed)
        transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed
    else:
        T_composed = np.eye(3)
        for i in range(anchor_idx + 1, moving_idx + 1):
            T_composed = np.dot(transformation_to_previous_sec[i], T_composed)
        transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed


warp_transforms = create_warp_transforms(animal, transformation_to_anchor_sec, downsample=True)
ordered_transforms = sorted(warp_transforms.items())
section_transform = {}

for section, transform in ordered_transforms:
    section_num = int(section.split('.')[0])
    transform = np.linalg.inv(transform)
    section_transform[section_num] = transform



100%|██████████| 447/447 [00:15<00:00, 29.41it/s]


In [29]:
##### Alignment of annotation coordinates
# Note, for missing sections, neuroglancer is minus one the section retrieved here
# Litao, check this code, most of it is yours but i added the defaultdict to fill up the points
keys = [k for k in structure_dict.keys()]
#Litao, this missing_sections will need to be manually built up from Beth's spreadhsheet
missing_sections = {k:[117] for k in keys}
fill_sections = defaultdict(dict)
other_structures = set()
volume = np.zeros((aligned_shape[1], aligned_shape[0], num_section), dtype=np.uint8)
for section in section_structure_vertices:
    template = np.zeros((aligned_shape[1], aligned_shape[0]), dtype=np.uint8)
    for structure in section_structure_vertices[section]:
        points = np.array(section_structure_vertices[section][structure])
        points = points / 32
        points = points + section_offset[section]  # create_clean offset
        points = points.reshape(1,2)
        points = transform_create_alignment(points, section_transform[section])  # create_alignment transform
        points = points.astype(np.int32)
        
        try:
            missing_list = missing_sections[structure]
        except:
            missing_list = []
            
        if section in missing_list:
            fill_sections[structure][section] = points
            
        try:
            #color = colors[structure.upper()]
            color = structure_dict[structure][1] # structure dict returns a list of [description, color]
            # for each key
        except:
            color = 255
            other_structures.add(structure)

        cv2.polylines(template, [points], True, color, 2, lineType=cv2.LINE_AA)
    volume[:, :, section - 1] = template

# fill up missing sections
template = np.zeros((aligned_shape[1], aligned_shape[0]), dtype=np.uint8)
for structure,v in fill_sections.items():
    color = structure_dict[structure][1]
    for section, points in v.items():
        cv2.polylines(template, [points], True, color, 2, lineType=cv2.LINE_AA)
        volume[:, :, section] = template

In [30]:
# Voxel resolution in nanometer (how much nanometer each element in numpy array represent)
resol = (14464, 14464, 20000)
# Voxel offset
offset = (0, 0, 0)
# Layer type
layer_type = 'segmentation'
# number of channels 
num_channels = 1
# segmentation properties in the format of [(number1, label1), (number2, label2) ...]
# where number is an integer that is in the volume and label is a string that describes that segmenetation

segmentation_properties = [(number, f'{structure}: {label}') for structure, (label, number) in structure_dict.items()]
extra_structures = ['Pr5', 'VTg', 'DRD', 'IF', 'MPB', 'Op', 'RPC', 'LSO', 'MVe', 'CnF', 
                    'pc', 'DTgC', 'LPB', 'Pr5DM', 'DTgP', 'RMC', 'VTA', 'IPC', 'DRI', 'LDTg', 
                    'IPA', 'PTg', 'DTg', 'IPL', 'SuVe', 'Sol', 'IPR', '8n', 'Dk', 'IO', 
                    'Cb', 'Pr5VL', 'APT', 'Gr', 'RR', 'InC', 'X', 'EW']
segmentation_properties += [(len(structure_dict) + index + 1, structure) for index, structure in enumerate(extra_structures)]

In [31]:
# https://github.com/seung-lab/cloud-volume
# Don't change all compress=False. It seems that Neuroglancer can only read with compress=False
from cloudvolume import CloudVolume
precompute_path = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/foundation_brain_annotations/{animal}'
cloudpath = f'file://{precompute_path}'
info = CloudVolume.create_new_info(
    num_channels = num_channels,
    layer_type = layer_type,
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = resol, # Voxel scaling, units are in nanometers
    voxel_offset = offset, # x,y,z offset in voxels from the origin
    chunk_size = [64, 64, 64], # units are voxels
    volume_size = volume.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

Uploading: 100%|██████████| 2464/2464 [00:25<00:00, 16.88it/s]

In [32]:
vol.info['segment_properties'] = 'names'
vol.commit_info()

segment_properties_path = os.path.join(precompute_path, 'names')
os.makedirs(segment_properties_path, exist_ok=True)

info = {
    "@type": "neuroglancer_segment_properties", 
    "inline": {
        "ids": [str(number) for number, label in segmentation_properties],
        "properties": [{
            "id": "label", 
            "description": "Name of structures",
            "type": "label",
            "values": [str(label) for number, label in segmentation_properties]
        }]
    }
}
with open(os.path.join(segment_properties_path, 'info'), 'w') as file:
    json.dump(info, file, indent=2)

In [33]:
# https://github.com/seung-lab/igneous
tq = LocalTaskQueue(parallel=True)
tasks = tc.create_downsampling_tasks(cloudpath, compress=False) # Downsample the volumes 
tq.insert(tasks)
tq.execute()


Tasks:   0%|          | 0/7 [00:00<?, ?it/s]

Volume Bounds:  Bbox([0, 0, 0],[1012, 1365, 447], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[1012, 1365, 447], dtype=int32)




Downloading:   0%|          | 0/352 [00:00<?, ?it/s]

Downloading:   0%|          | 0/352 [00:00<?, ?it/s]

Downloading:   8%|▊         | 29/352 [00:00<00:01, 216.50it/s]

Downloading:  10%|▉         | 35/352 [00:00<00:01, 216.50it/s]

Downloading:  14%|█▍        | 49/352 [00:00<00:01, 184.73it/s]

Downloading:  14%|█▍        | 50/352 [00:00<00:01, 184.73it/s]

Downloading:  19%|█▉        | 67/352 [00:00<00:02, 97.08it/s] 

Downloading:  24%|██▎       | 83/352 [00:00<00:03, 75.39it/s]

Downloading:  26%|██▌       | 91/352 [00:00<00:04, 61.24it/s]

Downloading:  28%|██▊       | 98/352 [00:00<00:04, 61.24it/s]

Downloading:  33%|███▎      | 117/352 [00:00<00:04, 50.30it/s]

Downloading:  36%|███▌      | 127/352 [00:00<00:05, 40.81it/s]

Downloading:  39%|███▉      | 139/352 [00:00<00:05, 40.81it/s]

Downloading:  44%|████▍     | 154/352 [00:00<00:04, 40.81it/s]

Downloading:  48%|████▊     | 169/352 [00:00<00:05, 31.90it/s]

Downloading:  51%|█████     | 178/352 [00:00<00:05, 31.90it/s

Uploading: 100%|██████████| 88/88 [00:01<00:00, 30.43it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00, 30.43it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00, 30.43it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00, 30.43it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00, 30.43it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00, 30.43it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|██████████| 88/88 [00:01<00:00,  7.73it/s]


Uploading: 100%|████████

Downloading: 100%|██████████| 352/352 [00:02<00:00, 38.26it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 38.26it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 38.26it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 21.16it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 21.16it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 21.16it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 21.16it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 21.16it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 21.16it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 38.26it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 38.26it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 21.16it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 38.26it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:00, 38.26it/s]





Downloading: 100%|██████████| 352/352 [00:02<00:

Uploading:  80%|███████▉  | 70/88 [00:02<00:19,  1.09s/it]




Uploading:  81%|████████  | 71/88 [00:02<00:19,  1.12s/it]




Uploading:  82%|████████▏ | 72/88 [00:02<00:18,  1.14s/it]




Uploading:  83%|████████▎ | 73/88 [00:02<00:17,  1.15s/it]




Uploading:  84%|████████▍ | 74/88 [00:03<00:16,  1.16s/it]




Uploading:  85%|████████▌ | 75/88 [00:03<00:15,  1.17s/it]

Uploading: 100%|██████████| 24/24 [00:01<00:00, 28.15it/s]




Uploading:  86%|████████▋ | 76/88 [00:03<00:14,  1.18s/it]

Uploading: 100%|██████████| 24/24 [00:01<00:00, 28.15it/s]




Uploading:  88%|████████▊ | 77/88 [00:03<00:13,  1.19s/it]




Uploading:  89%|████████▊ | 78/88 [00:03<00:10,  1.05s/it]




Uploading:  90%|████████▉ | 79/88 [00:03<00:08,  1.05it/s]




Uploading:  91%|█████████ | 80/88 [00:03<00:07,  1.14it/s]




Uploading:  92%|█████████▏| 81/88 [00:03<00:05,  1.23it/s]




Uploading:  93%|█████████▎| 82/88 [00:03<00:04,  1.30it/s]




Uploading:  94%|█████████▍| 83/88 [00:03<00:03,  1.36it/s]




Downloading:  79%|███████▊  | 277/352 [00:01<00:03, 20.18it/s]



Downloading:  83%|████████▎ | 293/352 [00:01<00:03, 18.68it/s]



Downloading:  86%|████████▌ | 302/352 [00:01<00:02, 18.68it/s]



Downloading:  89%|████████▉ | 315/352 [00:01<00:02, 16.02it/s]



Downloading:  93%|█████████▎| 326/352 [00:01<00:01, 14.46it/s]



Downloading:  95%|█████████▌| 336/352 [00:01<00:01, 11.35it/s]



Downloading:  97%|█████████▋| 342/352 [00:01<00:00, 11.35it/s]



Downloading:  99%|█████████▊| 347/352 [00:01<00:00, 10.14it/s]



Downloading: 100%|██████████| 352/352 [00:01<00:00,  9.06it/s]



Downloading: 100%|██████████| 352/352 [00:01<00:00,  9.06it/s]



Downloading: 100%|██████████| 352/352 [00:01<00:00,  9.06it/s]



Downloading: 100%|██████████| 352/352 [00:01<00:00,  9.06it/s]



Downloading: 100%|██████████| 352/352 [00:01<00:00,  9.06it/s]



Downloading: 100%|██████████| 352/352 [00:01<00:00,  9.06it/s]



Downloading: 100%|██████████| 352/352 [00:01<00:00,  9.06it/s]



Downloadin

Uploading: 100%|██████████| 24/24 [00:01<00:00, 21.40it/s]





Uploading: 100%|██████████| 24/24 [00:01<00:00, 21.40it/s]





Uploading:   0%|          | 0/24 [00:00<?, ?it/s]






Uploading:   0%|          | 0/6 [00:00<?, ?it/s]





Uploading:  50%|█████     | 12/24 [00:00<00:00, 80.04it/s]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33it/s]





Tasks:  71%|███████▏  | 5/7 [00:22<00:08,  4.28s/it]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33it/s]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33it/s]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33it/s]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33it/s]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33it/s]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33it/s]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33it/s]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33it/s]





Uploading: 100%|██████████| 24/24 [00:00<00:00, 137.33i

Uploading: 100%|██████████| 24/24 [00:00<00:00, 201.58it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 201.58it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 201.58it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 201.58it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 201.58it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 201.58it/s]

Uploading: 100%|██████████| 24/24 [00:00<00:00, 201.58it/s]


Tasks: 100%|██████████| 7/7 [00:26<00:00,  3.81s/it]


In [34]:
volume = np.swapaxes(volume, 0, 1)
all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=[14464, 14464, 20000]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer
print(viewer)

http://127.0.0.1:43189/v/ab152b65f015aeb1df2d65ae4d72840592a95fe1/
